In [13]:
from makiflow.layers import *
from makiflow.models.segmentation.segmentator import Segmentator
from makiflow.augmentation import AffineAugment, ElasticAugment, ImageCutter, Data
from makiflow.save_recover import Builder
from makiflow.trainers import SegmentatorTrainer
import makiflow as mf

import tensorflow as tf
import numpy as np
import glob
import cv2
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt

from sklearn.utils import shuffle
from scipy.ndimage import gaussian_filter

In [14]:
def load_data(path_to_data='../dataset/mask'):
    Xtrain = []
    Ytrain = []

    masks = glob.glob(f'{path_to_data}/*.bmp')
    for mask_name in tqdm(masks):
        img = cv2.imread(mask_name.replace('mask', 'imgs'))
        mask = cv2.imread(mask_name)
        Xtrain.append(img)
        Ytrain.append(mask)
        
    return Xtrain, Ytrain

In [15]:
def calc_num_pos(labels):
    area = labels[0].shape[0] * labels[0].shape[1]
    return [area - (label == 0).sum() for label in labels]

### Load data

In [16]:
images, labels = load_data()

100%|██████████| 100/100 [00:00<00:00, 183.12it/s]


In [34]:
Xtest = []
Ytest = []
Xtrain = []
Ytrain = []

count = 0
for i, (img, lbl) in enumerate(zip(images, labels)):
    u = np.unique(lbl)
    if 60 in u or 40 in u or 70 in u or 80 in u:
        print(f'{i}: {u}')
        count += 1
        Xtest.append(img)
        Ytest.append(lbl)
print(count)

1: [ 0 10 20 30 50 60 90]
2: [ 0 10 20 30 40 50 90]
5: [ 0 10 20 30 40 50 60 80 90]
6: [ 0 10 20 30 50 60 90]
7: [ 0 10 20 30 50 60 90]
11: [ 0 10 20 30 50 70 90]
12: [ 0 10 20 30 50 60 90]
13: [ 0 10 20 30 50 60 90]
15: [ 0 10 20 30 50 60 70 90]
17: [ 0 10 20 30 50 60 90]
20: [ 0 10 20 30 40 50 90]
22: [ 0 10 20 30 50 60 90]
24: [ 0 10 20 30 50 80 90]
26: [ 0 10 20 30 50 60 90]
27: [ 0 10 20 30 50 60 70 90]
28: [ 0 10 20 30 50 60 90]
29: [ 0 10 20 30 50 60 90]
30: [ 0 10 20 30 50 80 90]
31: [ 0 10 20 30 50 60 90]
32: [ 0 10 20 30 40 50 80 90]
36: [ 0 10 20 30 50 60 80 90]
37: [ 0 10 20 30 50 60 90]
39: [ 0 10 20 30 50 60 90]
41: [ 0 10 20 30 50 70 90]
42: [ 0 10 20 30 50 60 90]
44: [ 0 10 20 30 40 90]
46: [ 0 10 20 30 50 60 90]
48: [ 0 10 20 30 50 60 90]
49: [ 0 10 20 30 50 60 90]
50: [ 0 10 20 30 50 60 90]
53: [ 0 10 20 30 40 50 90]
54: [ 0 10 20 30 40 50 90]
56: [ 0 10 20 30 40 60 80 90]
60: [ 0 10 20 30 50 80 90]
62: [ 0 10 20 30 50 60 90]
63: [ 0 10 20 30 50 80 90]
65: [ 0 10 20 3

In [ ]:
2 # 4
24 # 8
41 # 7
85 # 8
75 # 4

In [ ]:
Xtest = []
Ytest = []
Xtrain = []
Ytrain = []

for i, label in enumerate(labels):
    uniq = np.unique(label)
    if i in []:
        Xtest.append(images[i])
        Ytest.append(label)
    else:
        Xtrain.append(images[i])
        Ytrain.append(label)

In [ ]:
print(len(Xtrain), len(Xtest))

# Important 
25/41/14/19/43/2/5

In [ ]:
from copy import copy

In [ ]:
for image, label in zip(copy(Xtrain), copy(Ytrain)):
    uniq = np.unique(label)
    if 40 in uniq or 70 in uniq or 80 in uniq:
        Ytrain += [label] * 10
        Xtrain += [image] * 10

In [ ]:
# Xtrain, Ytrain, _ = ImageCutter.image_and_mask_cutter(Xtrain, Ytrain, 256, 256, 128, 128, 0.5)
Xtest, Ytest, _ = ImageCutter.image_and_mask_cutter(Xtest, Ytest, 256, 256, 128, 128, 0.5)
print(len(Xtrain), len(Xtest))

In [ ]:
Xtrain = np.asarray(Xtrain).astype(np.float32) / 255
Xtrain = [i for i in Xtrain]
Ytrain = np.asarray(Ytrain).astype(np.uint8) // 10
Ytrain = [i for i in Ytrain]
Xtest = np.asarray(Xtest).astype(np.float32) / 255
Xtest = [i for i in Xtest]
Ytest = np.asarray(Ytest).astype(np.uint8) // 10
Ytest = [i for i in Ytest]

In [ ]:
data = Data(Xtrain, Ytrain)
data = AffineAugment(num_matrices=2, noise_type='gaussian', keep_old_data=True)(data)
data = ElasticAugment(num_maps=3, border_mode='reflect_101',  keep_old_data=True)(data)

In [ ]:
aug_images, aug_labels = data.get_data()
print(len(aug_images))
print(type(aug_images))

In [ ]:
Ytrain = [cv2.cvtColor(item, cv2.COLOR_BGR2GRAY) for item in Ytrain]


In [ ]:
aug_labels = [cv2.cvtColor(item, cv2.COLOR_BGR2GRAY) for item in aug_labels]


In [ ]:
Ytest = [cv2.cvtColor(item, cv2.COLOR_BGR2GRAY) for item in Ytest]

In [ ]:
num_pos = calc_num_pos(Ytrain)

In [ ]:
sum([i > 0 for i in num_pos]) == len(num_pos)

In [ ]:
print(Xtest[0].shape, Ytest[0].shape)

In [ ]:
mf.set_main_gpu(0)
model = Builder.segmentator_from_json('unetMobileNet/model.json')


In [ ]:
model.set_session(tf.Session())

In [ ]:
model.load_weights('result/Test_pretrained_Unet_with_MobileNetV2_backbone/MakiSegmentator_gamma=2.0_lr=0.001_bsz=32/last_weights/weights.ckpt')

In [ ]:
from sklearn.metrics import confusion_matrix
def confusion_mat(
        p, l,
        use_argmax_p=False, use_argmax_l=False, to_flatten=False, normalize=True,
        save_path=None, dpi=150, annot=False):
    """
    Creates confusion matrix for the given predictions `p` and labels `l`.
    Parameters
    ----------
    p : np.ndarray
        Predictions.
    l : np.ndarray
        Corresponding labels.
    use_argmax_p : bool
        Set to true if prediction aren't sparse, i.e. `p` is an array of shape [..., num_classes].
    use_argmax_l : bool
        Set to True if labels aren't sparse (one-hot encoded), i.e. `l` is an array of shape [..., num_classes].
    to_flatten : bool
        Set to True if `p' and `l` are high-dimensional arrays.
    normalize : bool 
        Set to True if you want to ge normalized matrix.
    save_path : str
        Saving path for the confusion matrix picture.
    dpi : int
        Affects the size of the saved confusion matrix picture.
    annot : bool
        Set to true if want to see actual numbers on the matrix picture.
    """
    if use_argmax_p:
        p = p.argmax(axis=-1)

    if use_argmax_l:
        l = l.argmax(axis=-1)

    if to_flatten:
        p = p.reshape(-1)
        l = l.reshape(-1)

    mat = np.asarray(confusion_matrix(l, p), dtype=np.float32)
    mat /= mat.sum(axis=0)
    mat = np.round(mat, decimals=2)
    del p
    del l

    if save_path is not None:
        conf_mat = sns.heatmap(mat, annot=annot)
        conf_mat.figure.savefig(save_path, dpi=dpi)
        plt.close(conf_mat.figure)
    return mat

In [ ]:
batch_sz = 16

In [ ]:
n_batches = len(Xtrain[:250]) // batch_sz
labels = []
predictions = []
for i in range(n_batches):
    labels += Ytrain[i * batch_sz: (i+1) * batch_sz]
    predictions += [model.predict(Xtrain[i * batch_sz: (i+1) * batch_sz])]

In [ ]:
n_batches = len(Xtest) // batch_sz
labels = []
predictions = []
for i in range(n_batches):
    labels += Ytest[i * batch_sz: (i+1) * batch_sz]
    predictions += [model.predict(Xtest[i * batch_sz: (i+1) * batch_sz])]

In [ ]:
predictions = np.vstack(predictions)
labels = np.asarray(labels)

In [ ]:
mat = confusion_mat(predictions, labels, use_argmax_p=True, to_flatten=True, save_path='mat.png', annot=True)
mat

In [ ]:
mat / mat.sum(axis=0)

In [ ]:
import glob

In [ ]:
glob.glob('../dataset/imgs/*')

In [17]:
import cv2


class ImageCutter:

    @staticmethod
    def image_and_mask_cutter(
        images, masks, window_h, window_w, step_x, step_y, classes_to_get,
        use_all_px=True
    ):
        """
        Crops `images` and `masks` using sliding window with resize.
        Parameters
        ----------
        images : list
            List of input images.
        masks : list
            List of input masks.
        window_h : int
            Output image height.
        window_w : int
            Output image width.
        step_x : int
            Sliding window step by OX.
        step_y : int
            Sliding window step by OX.
        scale_factor : float
            Scale factor, must be in range (0, 1). After each 'sliding window step' the original images
            are resized to (previous_width * scale_factor, previous_height * scale_factor).
        postprocessing : func
            Post processing function, using on cropped image (may be function what calculate num positives pixels).
        use_all_px : bool
            If True, all pixels of image would be in output lists.

        Returns
        -------
        Three list:
            1. cropped images
            2. cropped masks
            3. additional list (result of post processing)
        """
        assert (len(images) > 0)
        assert (len(images) == len(masks))
        assert (window_h > 0 and window_w > 0 and step_x > 0 and step_y > 0)

        cropped_images = []
        cropped_masks = []
        additional_list = []
        dx = 0
        dy = 0

        for index, (img, mask) in enumerate(zip(images, masks)):
            print(index)
            assert (img.shape[:2] == mask.shape[:2])
            current_height, current_width = img.shape[:2]


            for dy in range(int((current_height - window_h) / step_y)):
                for dx in range(int((current_width - window_w) / step_x)):
                    crop_img, crop_mask = ImageCutter.crop_img_and_mask(
                        img,
                        mask,
                        dy * step_y, dy * step_y + window_h, dx * step_x, dx * step_x + window_w)
                    if ImageCutter.has_class(crop_mask, classes_to_get):
                        cropped_images.append(crop_img)
                        cropped_masks.append(crop_mask)

            if use_all_px:
                overlap_y = dy * step_y + window_h != current_height
                overlap_x = dx * step_x + window_w != current_width
                if overlap_y:
                    for dx in range(int((current_width - window_w) / step_x)):
                        crop_img, crop_mask = ImageCutter.crop_img_and_mask(
                            img,
                            mask,
                            current_height - window_h, current_height, dx * step_x, dx * step_x + window_w)
                        if ImageCutter.has_class(crop_mask, classes_to_get):
                            cropped_images.append(crop_img)
                            cropped_masks.append(crop_mask)

                if overlap_x:
                    for dy in range(int((current_height - window_h) / step_y)):
                        crop_img, crop_mask = ImageCutter.crop_img_and_mask(
                            img,
                            mask,
                            dy * step_y, dy * step_y + window_h, current_width - window_w, current_width)
                        if ImageCutter.has_class(crop_mask, classes_to_get):
                            cropped_images.append(crop_img)
                            cropped_masks.append(crop_mask)

                if overlap_x and overlap_y:
                    crop_img, crop_mask = ImageCutter.crop_img_and_mask(
                        img,
                        mask,
                        current_height - window_h, current_height, current_width - window_w, current_width)
                    if ImageCutter.has_class(crop_mask, classes_to_get):
                        cropped_images.append(crop_img)
                        cropped_masks.append(crop_mask)

        return cropped_images, cropped_masks, additional_list

    @staticmethod
    def crop_img_and_mask(img, mask, up, down, left, right):
        crop_img = img[up: down, left: right]
        crop_mask = mask[up: down, left: right]
        return crop_img, crop_mask
    
    @staticmethod
    def has_class(mask, needed):
        actual = np.unique(mask)
        for need in needed:
            if need in actual:
                return True
        return False


In [44]:
imgs, lbls = [], []
for i, (img, lbl) in enumerate(zip(images, labels)):
    if i in [2, 24, 41, 85, 75, 6, 7]:
        imgs += [img]
        lbls += [lbl]

In [36]:
2 # 4
24 # 8
41 # 7
85 # 8
75 # 4
6 # 6

75

In [45]:
#imgs, lbls = load_data()

cropped_images1, cropped_masks1, _ = ImageCutter.image_and_mask_cutter(
    imgs, lbls, window_h=256, window_w=256, step_x=50, step_y=50, classes_to_get=[40, 70, 80],
)

0
1
2
3
4
5
6


In [46]:
#imgs, lbls = load_data()
# 7
cropped_images2, cropped_masks2, _ = ImageCutter.image_and_mask_cutter(
    imgs, lbls, window_h=256, window_w=256, step_x=40, step_y=40, classes_to_get=[70],
)

0
1
2
3
4
5
6


In [47]:
#imgs, lbls = load_data()
# 6
cropped_images3, cropped_masks3, _ = ImageCutter.image_and_mask_cutter(
    imgs, lbls, window_h=256, window_w=256, step_x=60, step_y=60, classes_to_get=[60],
)

0
1
2
3
4
5
6


In [48]:
#imgs, lbls = load_data()
# 10
cropped_images4, cropped_masks4, _ = ImageCutter.image_and_mask_cutter(
    imgs, lbls, window_h=256, window_w=256, step_x=60, step_y=60, classes_to_get=[10],
)

0
1
2
3
4
5
6


In [ ]:
# 10
class_dict = {i*10: 0 for i in range(10)}
for index, mask in enumerate(cropped_masks1):
    uniq = np.unique(mask)
    for k in uniq:
        class_dict[k] += 1

In [ ]:
class_dict

In [ ]:
# 70
class_dict = {i*10: 0 for i in range(10)}
for index, mask in enumerate(cropped_masks2):
    uniq = np.unique(mask)
    for k in uniq:
        class_dict[k] += 1

In [ ]:
class_dict

In [ ]:
# 60
class_dict = {i*10: 0 for i in range(10)}
for index, mask in enumerate(cropped_masks3):
    uniq = np.unique(mask)
    for k in uniq:
        class_dict[k] += 1

In [ ]:
class_dict

In [ ]:
# 60
class_dict = {i*10: 0 for i in range(10)}
for index, mask in enumerate(cropped_masks4):
    uniq = np.unique(mask)
    for k in uniq:
        class_dict[k] += 1

In [ ]:
class_dict

In [ ]:
# 10
class_dict = {i*10: 0 for i in range(10)}
for index, mask in enumerate(cropped_masks5):
    uniq = np.unique(mask)
    for k in uniq:
        class_dict[k] += 1

In [ ]:
class_dict

In [ ]:
class_dict = {i*10: 0 for i in range(10)}
for index, mask in enumerate(cropped_masks3+cropped_masks2+cropped_masks1+cropped_masks):
    uniq = np.unique(mask)
    for k in uniq:
        class_dict[k] += 1

In [ ]:
class_dict

In [50]:
class_dict = {i*10: 0 for i in range(10)}
for index, mask in enumerate(cropped_masks3+cropped_masks2+cropped_masks1+cropped_masks4):
    uniq = np.unique(mask)
    for k in uniq:
        class_dict[k] += 1

In [51]:
class_dict

{0: 726,
 10: 176,
 20: 177,
 30: 726,
 40: 208,
 50: 413,
 60: 85,
 70: 18,
 80: 274,
 90: 672}

In [63]:
len(cropped_masks3+cropped_masks2+cropped_masks1+cropped_masks4 + cropped_masks1[:10])

736

In [65]:
736 % 32

0

In [59]:
len(cropped_masks1)

473